In [1]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc, shutil

from collections import Counter
import itertools

from eventsSuggester import ClicksSuggester, CartsSuggester, OrdersSuggester, BuysSuggester

In [2]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'

In [3]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

In [4]:
TOPN_clicks = 20
TOPN_b2b = 20
TOPN_cartsOrders = 20
candidatesNum = 50
SET = 2
VER = 6
# note_covisit = 'covisit_20_20_20'
note_candidate = 'covisit_20_20_20_covWgtSuggester'
predTypes = ['clicks', 'carts', 'orders']

In [6]:
def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob('../input/parquets/test_parquets/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

In [6]:
def addTopEvent(x, topN, topEvents):
    idx = 0
    resultSet = set(x)
    while(len(x) < topN):
        if topEvents[idx] not in resultSet:
            x.append(topEvents[idx])
        idx += 1
    return x

In [7]:
SETS = [2]
for SET in SETS:
    if SET == 1:
        testAPath = inputPath + 'val_A.parquet'
        testA = pd.read_parquet(testAPath)

    elif SET == 2:
        testAPath = '../input/splited/test.parquet'
        testA = pd.read_parquet(testAPath)

    elif SET == 3:
        testA = load_test()
    coVisitSaveFolder = outputPath + f'coVisit_set{SET}_top{TOPN_b2b}_{TOPN_clicks}_{TOPN_cartsOrders}/'
    
    top_clicks = testA.loc[testA['type']==0,'aid'].value_counts().index.values[:candidatesNum].astype(np.int32)
    top_carts = testA.loc[testA['type'] == 1,'aid'].value_counts().index.values[:candidatesNum].astype(np.int32)
    top_orders = testA.loc[testA['type'] == 2,'aid'].value_counts().index.values[:candidatesNum].astype(np.int32)

    candidatesSavePath = outputPath + f'candidates_set{SET}_top{candidatesNum}_{note_candidate}/'
    ! mkdir {candidatesSavePath}

    for predType in predTypes:
        type2covisitType = {'clicks':'clicks', 'carts': 'cartsOrders', 'orders': 'cartsOrders'}
        for k in range(0, 8):
            if k == 0:
                covistWeight = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_{type2covisitType[predType]}_v{VER}_{k}.pqt')
            else:
                covistWeight = pd.concat([covistWeight, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_{type2covisitType[predType]}_v{VER}_{k}.pqt')], axis=0)
        uniqTestA = testA.drop_duplicates(['session', 'aid'])
        uniqTestA.columns = ['session', 'aid_x', 'ts', 'type']
        df = uniqTestA.merge(covistWeight, on='aid_x', how='left')

        df = df.groupby(['session', 'aid_y']).agg({'wgt':'sum'}).reset_index()
        df = df.sort_values(['session', 'wgt'], ascending=[True,False]).reset_index(drop=True)
        df['n'] = df.groupby('session').cumcount()
        uniqTestA = uniqTestA[['session', 'aid_x']]
        uniqTestA.columns = ['session', 'aid']
        uniqTestA['ranking'] = -1
        df = df[['session', 'aid_y', 'n']]
        df.columns = ['session', 'aid', 'ranking']
        df['aid'] = df['aid'].astype(int)
        df = pd.concat([uniqTestA, df]).drop_duplicates(['session', 'aid']).sort_values(['session', 'ranking'], ascending=[True, True]).reset_index(drop=True)
        df['ranking'] = df.groupby('session').cumcount()
        df = df[df['ranking']<candidatesNum]
        df = df.groupby('session').aid.apply(list).reset_index()
        df['aid'] = df.aid.apply(lambda x: addTopEvent(x, candidatesNum, top_clicks))
        aids = df.aid.explode().astype('int32')
        candidate = df[['session']].astype('int32')
        candidate = candidate.merge(aids, left_index=True, right_index=True, how='left').reset_index(drop=True)
        candidate.to_parquet(candidatesSavePath + f'{predType}Candidate.pqt')

mkdir: cannot create directory ‘../output/newSplited/candidates_set2_top50_covisit_20_20_20_covWgtSuggester/’: File exists


# Calculate recall (SET == 2)

In [5]:
SET = 2
note_candidate = 'covisit_20_20_20_newCartSuggester'

In [6]:
candidatesSavePath = outputPath + f'candidates_set2_top{candidatesNum}_{note_candidate}/'

In [7]:
toConcat = []
for predType in predTypes:
    pred_df = pd.read_parquet(candidatesSavePath + f'{predType}Candidate.pqt')
    pred_df = pred_df.groupby('session').aid.apply(list).reset_index()
    pred_df.columns = ['session', 'labels']
    pred_df['type'] = predType
    toConcat.append(pred_df)
pred_df = pd.concat(toConcat).reset_index(drop=True)
pred_df

,session,labels,type
0,11098528,"[11830, 588923, 1732105, 571762, 884502, 11578...",clicks
1,11098529,"[1105029, 459126, 1339838, 1544564, 217742, 16...",clicks
2,11098530,"[409236, 264500, 1603001, 963957, 254154, 5830...",clicks
3,11098531,"[396199, 1271998, 452188, 1728212, 1365569, 62...",clicks
4,11098532,"[876469, 7651, 108125, 1202618, 1159379, 77906...",clicks
...,...,...,...
5403748,12899774,"[33035, 819288, 1539309, 95488, 743977, 270852...",orders
5403749,12899775,"[1743151, 1760714, 1498443, 1255910, 1163166, ...",orders
5403750,12899776,"[548599, 1440959, 1144446, 695829, 1401030, 16...",orders
5403751,12899777,"[384045, 1308634, 1688215, 395762, 1486067, 70...",orders


In [8]:
if SET == 1:
    test_label_path = '../input/newSplited/val_B.parquet'
    test_labels = pd.read_parquet(test_label_path)
elif SET == 2:
    test_label_path = '../input/splited/test_labels.parquet'
    test_labels = pd.read_parquet(test_label_path)

In [9]:
test_labels = test_labels.merge(pred_df, how='left', on=['session', 'type'])
test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels[:20]))), axis=1)
test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
test_labels

,session,type,ground_truth,labels,hits,gt_count
0,11098528,clicks,[1679529],"[11830, 588923, 1732105, 571762, 884502, 11578...",0,1
1,11098528,carts,[1199737],"[11830, 1732105, 588923, 884502, 876129, 11578...",0,1
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369...","[11830, 1732105, 588923, 884502, 876129, 11578...",1,11
3,11098529,clicks,[1105029],"[1105029, 459126, 1339838, 1544564, 217742, 16...",1,1
4,11098530,orders,[409236],"[409236, 264500, 1603001, 963957, 254154, 3641...",1,1
...,...,...,...,...,...,...
2212687,12899774,clicks,[1399483],"[33035, 1539309, 819288, 95488, 771913, 270852...",1,1
2212688,12899775,clicks,[1760714],"[1743151, 1760714, 1255910, 1163166, 832192, 2...",1,1
2212689,12899776,clicks,[1737908],"[548599, 1401030, 1440959, 1607333, 1144446, 3...",0,1
2212690,12899777,clicks,[384045],"[384045, 1308634, 1688215, 703474, 395762, 148...",1,1


In [10]:
print('Set:', SET, 'Top:', candidatesNum)
recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum()
print(recall_per_type)
score = (recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()
print(score)

Set: 2 Top: 50
type
carts     0.406391
clicks    0.525474
orders    0.635774
dtype: float64
0.5559292684072807
